# Patch loader experimentation notebook

The patch loader is going to be our harness **entry point**. Where we are going to **load**, **apply**, and **manage** LLM-generated patches to repositories.
First of all, what is a patch?

A patch is a textual representation of code changes, basically a diff showing what lines were added, removed, or modified in a repository.
When an LLM-based agent tries to solve a **SWE-bench task**, it receives , for example, something like:

“Here is a failing test and the project context. Please modify the code so the test passes."

The agent reads the given code and generates a *patch* doing a *diff* operation between the old and new code strings
```text
diff --git a/app/utils.py b/app/utils.py
index 3a4bcd2..e8f7aab 100644
--- a/app/utils.py
+++ b/app/utils.py
@@ -42,7 +42,10 @@ def get_value(key):
     value = cache.get(key)
-    if value:               # "-" indicates the old code
-        return value
-    return None
+    if value is not None:   # "+" indicates the generated code from the agent to solve the issue"
+        return value
+    else:
+        logger.warning(f"Cache miss for {key}")
+        return None
´´´



In [3]:
from datasets import load_dataset
ds = load_dataset("princeton-nlp/SWE-bench_Verified", split="test")

# Example entry
sample = ds[0]
sample  # shows the beginning of the diff

{'repo': 'astropy/astropy',
 'instance_id': 'astropy__astropy-12907',
 'base_commit': 'd16bfe05a744909de4b27f5875fe0d4ed41ce607',
 'patch': "diff --git a/astropy/modeling/separable.py b/astropy/modeling/separable.py\n--- a/astropy/modeling/separable.py\n+++ b/astropy/modeling/separable.py\n@@ -242,7 +242,7 @@ def _cstack(left, right):\n         cright = _coord_matrix(right, 'right', noutp)\n     else:\n         cright = np.zeros((noutp, right.shape[1]))\n-        cright[-right.shape[0]:, -right.shape[1]:] = 1\n+        cright[-right.shape[0]:, -right.shape[1]:] = right\n \n     return np.hstack([cleft, cright])\n \n",
 'test_patch': "diff --git a/astropy/modeling/tests/test_separable.py b/astropy/modeling/tests/test_separable.py\n--- a/astropy/modeling/tests/test_separable.py\n+++ b/astropy/modeling/tests/test_separable.py\n@@ -28,6 +28,13 @@\n p1 = models.Polynomial1D(1, name='p1')\n \n \n+cm_4d_expected = (np.array([False, False, True, True]),\n+                  np.array([[True,  Tr

**SWE-bench** and its variations already provide generated patches from many models such as GPT,Claude,Gemini... We will be using those in order to test that our pipeline works correctly

**DISCLAIMER** The harness that we are building is *model-agnostic* -> It judges any proposed patch on its technical merit alone. 

In the [repo](https://github.com/SWE-bench/experiments/tree/main/evaluation) the overal results of the models are stored. It will be useful for later evaluation

So the patch loader is in charge of taking that string and:

* Loads the base repo (same commit SWE-bench uses).

* Applies the patch via git apply.

* Returns the patched repo so the rest of your modules can:

    * run static checks (flake8, mypy),

    * run dynamic verification (pytest, fuzzing, coverage),

    * and finally see if the issue is fixed